In [ ]:
#  Turn in inline presentations
%matplotlib

In [ ]:
# Imports
import os
import time
import gdal
import pandas as pd

import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.patches as patches # Needed to draw rectangles
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
os.chdir('/Users/jamesash/documents/Salo_ai/SAR_CH3/')
if os.path.isdir('wa_v2') == False:
    # output_dir = '/private/tmp'
    output_dir = '/Users/jamesash/documents/Salo_ai/SAR_CH3/'
    !wget https://gis1.servirglobal.net/TrainingMaterials/SAR/Data_ch3_wa.zip
    !unzip Data_ch3_wa.zip
    file_list = glob('Data_ch3_wa/*')
    for files in file_list:
        os.rename(file, output_dir + os.path.basename(file))
else:
    print("Files exist in directory")

In [ ]:
datadirectory = os.getcwd() + '/wa_v2/BIOsS1'
datefile = 'S32631X398020Y1315440sS1_A_vv_0001_mtfil.dates'
imagefile='S32631X398020Y1315440sS1_A_vv_0001_mtfil.vrt'
subset = None

In [ ]:
os.chdir(datadirectory)

In [ ]:
# Get the date indices via pandas
dates = open(datefile).readlines()
tindex = pd.DatetimeIndex(dates)

In [ ]:
tindex

In [ ]:
#  Open the image and read the first raster band
img = gdal.Open(imagefile)
band = img.GetRasterBand(1)
# Set the subset
if subset == None:
    subset = (0, 0, img.RasterXSize, img.RasterYSize)

In [ ]:
subset

In [ ]:
# Plot one band and subset outline to see which subset we are intersetedd in 
raster = band.ReadAsArray()
vmin = np.percentile(raster.flatten(), 5)
vmax = np.percentile(raster.flatten(), 95)
fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(111)
ax.imshow(raster, cmap = 'gray', vmin = vmin, vmax = vmax)
# Plot the subset as rectangle
_ = ax.add_patch(patches.Rectangle((subset[0], subset[1]), subset[2], subset[3], 
                                 fill = False, edgecolor = 'red'))

In [ ]:
raster0 = band.ReadAsArray(*subset)
bandnbr = 0 # Needed for updates
rasterstack = img.ReadAsArray(*subset)

In [ ]:
%%capture
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np


fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(111)
ax.axis('off')
vmin = np.percentile(rasterstack.flatten(), 5)
vmax = np.percentile(rasterstack.flatten(), 95)

im = ax.imshow(raster0, cmap = 'gray', vmin = vmin, vmax = vmax)
ax.set_title("{}".format(tindex[0].date()))

def animate(i):
    ax.set_title("{}".format(tindex[i].date()))
    im.set_data(rasterstack[i])

# Interval is given in milliseconds
ani = matplotlib.animation.FuncAnimation(fig, animate,
                                        frames = rasterstack.shape[0],
                                        interval = 400)


In [ ]:
rc('animation', embed_limit = 40971520.0) # We need to increase the limit to show the entire animation

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
# 1. Conversion to Power
caldB = -83
calPwr = np.power(10.,caldB/10.)
rasterstack_pwr = np.power(rasterstack, 2.)*calPwr

# Compute Means
rs_means_pwr = np.mean(rasterstack_pwr, axis = (1, 2))

#  Convert to dB
rs_means_dB = 10.*np.log10(rs_means_pwr)

In [ ]:
rs_means_pwr.shape # Check that we got the means over time

In [ ]:
fig = plt.figure(figsize = (16,4))
ax1 = fig.add_subplot(111)
ax1.plot(tindex, rs_means_pwr)
ax1.set_xlabel('Date')
ax1.set_ylabel('$\overline{\gamma^o}$ [power]')

ax2 = ax1.twinx()
ax2.plot(tindex, rs_means_dB, color = 'red')
ax2.set_ylabel('$\overline{\gamma^o}$ [power]')
fig.legend(['Power','dB'], loc = 1)
plt.title('Time Series profile of average band backscatter $\gamma^o$')

In [ ]:
a = pd.Series(rs_means_dB, index = tindex)

In [ ]:
a

In [ ]:
%%capture
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (16, 4), gridspec_kw = {'width_ratios': [1, 3]})

vmin = np.percentile(rasterstack.flatten(), 5)
vmax = np.percentile(rasterstack.flatten(), 95)
im = ax1.imshow(raster0, cmap = 'gray', vmin = vmin, vmax = vmax)
ax1.set_title("{}".format(tindex[0].date()))
ax1.set_axis_off()

ax2.axis([tindex[0], tindex[-1], rs_means_dB.min(), rs_means_dB.max()])
ax2.set_ylabel('$\overline{\gamma^o}$ [dB]')
ax2.set_xlabel('Date')
ax2.set_ylim((-5, -15))
l, = ax2.plot([],[])

def animate(i):
    ax1.set_title("{}".format(tindex[i].date()))
    im.set_data(rasterstack[i])
    ax2.set_title("{}".format(tindex[i].date()))
    l.set_data(tindex[:(i+1)], rs_means_dB[:(i+1)])
    
ani = matplotlib.animation.FuncAnimation(fig, animate,
                                        frames = rasterstack.shape[0],
                                        interval = 400)

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())